In [18]:
# import packages
import pandas as pd
import json
import osmnx as ox
import requests
import random
import time

In [19]:
# function to generate random lat lon coords within a closed range
def random_coords(center_coords, count):
    lat = []
    lon = []
    
    for i in range(count):
        #set random generator
        lat.append(random.uniform(center_coords[0]-0.1, center_coords[0]+0.1))
        lon.append(random.uniform(center_coords[1]-0.1, center_coords[1]+0.1))
    
    return lat, lon

In [20]:
# determine center coords to be downtown Seattle
center_coords = [47.608013, -122.335167]

lat, lon = random_coords(center_coords, 1000)

In [ ]:
# set empty walk_score list
walk_score = []

apikey = 'APIKEY'



for i in range(len(lat)):
    url = 'http://api.walkscore.com/score?format=json&lat='+str(lat[i])+'&lon='+str(lon[i])+'&wsapikey='+apikey
    r = requests.get(url)
    data = r.json()
    if 'walkscore' in data:
        walk_score.append(data['walkscore'])
    else:
        walk_score.append('N/A')
        
    time.sleep(0.2)    

In [ ]:
# create dataframe
new = pd.DataFrame(list(zip(lat, lon, walk_score)),  columns=['lat', 'lon', 'walk_score'])
new.info()

In [ ]:
GEOID = []
for i in range(len(new)):
    url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x='+str(new.lon[i])+'&y='+str(new.lat[i])+'&benchmark=Public_AR_ACS2018&vintage=Current_ACS2018&format=json'
    r = requests.get(url)
    data = r.json()
    if 'GEOID' in data['result']['geographies']['Census Tracts'][0]:
        tag = data['result']['geographies']['Census Tracts'][0]['GEOID']
    else:
        tag = 'N/A'
        
    GEOID.append(tag)
    
new['GEOID'] = GEOID

In [ ]:
new = new[new['GEOID'] != 'N/A']
new = new.dropna()

new['GEOID'] = new['GEOID'].astype(int)

new = new.reset_index(drop=True)

In [ ]:
G = ox.graph_from_place('Seattle, Washington', network_type='walk')
ox.plot_graph(G)

In [ ]:
# number of intersections in graph, that is, nodes with >1 street emanating from them
intersection_count = []

#how many streets (edges in the undirected representation of the graph) emanate from each node (ie,intersection or dead-end) on average (mean)
streets_per_node_avg = []

# edge_length_total divided by the sum of the great circle distances between the nodes of each edge
circuity_avg = []

# mean edge length in the undirected representation of the graph, in meters
street_length_avg = []


## Street network within 0.25km of the lat-lon point
for i in range(len(new['lat'])):
    try:
        G = ox.graph_from_point((new['lat'][i], new['lon'][i]), distance=250, network_type='walk')
        stats = ox.basic_stats(G)
        intersection_count.append(stats['intersection_count'])
        streets_per_node_avg.append(stats['streets_per_node_avg'])
        circuity_avg.append(stats['circuity_avg'])
        street_length_avg.append(stats['street_length_avg'])
    except:
        intersection_count.append(0)
        streets_per_node_avg.append(0)
        circuity_avg.append(0)
        street_length_avg.append(0)
        
new['intersection_count'] = intersection_count
new['streets_per_node_avg'] = streets_per_node_avg
new['circuity_avg'] = circuity_avg
new['street_length_avg'] = street_length_avg  

In [ ]:
apikey = apikey

def location_api(df, amenity, radius, apikey):
    amenity_count = [] 
    for i in range(len(df)):
        count = 0    
        url = 'https://us1.locationiq.com/v1/nearby.php?key='+apikey+'&lat='+str(df['lat'][i])+'&lon='+str(df['lon'][i])+'&tag='+amenity+'&radius='+radius+'&format=json'
        r = requests.get(url)
        data = r.json()
        if 'error' in data:
            amenity_count.append(0)
            time.sleep(0.62)
        else:   
            count = len(data)
            amenity_count.append(count)
            time.sleep(0.75)

    df[amenity+'_count'] = amenity_count
    #df[amenity+'wide_count'] = amenity_count
    
    return df

In [ ]:
new.info()

In [ ]:
new = location_api(new, 'restaurant', '1000', private_token);
new = location_api(new, 'school', '1000', private_token);
new = location_api(new, 'park', '1000', private_token);
new = location_api(new, 'bus_station', '10000', private_token);
new = location_api(new, 'supermarket', '1000', private_token);
new = location_api(new, 'pub', '1000', private_token);

In [ ]:
apikey=apikey

def location_api(df, amenity, radius, apikey):
    amenity_count = [] 
    for i in range(len(df)):
        count = 0    
        url = 'https://us1.locationiq.com/v1/nearby.php?key='+apikey+'&lat='+str(df['lat'][i])+'&lon='+str(df['lon'][i])+'&tag='+amenity+'&radius='+radius+'&format=json'
        r = requests.get(url)
        data = r.json()
        if 'error' in data:
            amenity_count.append(0)
            time.sleep(0.62)
        else:   
            count = len(data)
            amenity_count.append(count)
            time.sleep(0.75)

    #df[amenity+'_count'] = amenity_count
    df[amenity+'wide_count'] = amenity_count
    
    return df

In [ ]:
new = location_api(new, 'park', '2500', private_token);
new = location_api(new, 'restaurant', '1500', private_token);

In [ ]:
new.info()

In [ ]:
new.to_csv('Data/master_walk_score.csv')

In [ ]:
new.shape